In [ ]:
%pip install azure-ai-ml python-dotenv azureml-core

In [38]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
import os
from azureml.core import Environment
from azureml.core.workspace import Workspace
from azure.ai.ml import command
from azure.ai.ml.entities import ComputeInstance

In [39]:
if os.path.exists("configs.env"):
    load_dotenv("configs.env")

In [40]:
subscription_id = os.getenv("SUBSCRIPTION_ID")
resource_group = os.getenv("RESOURCE_GROUP")
workspace = os.getenv("WORKSPACE_NAME")

In [41]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [42]:
ci_basic_name = "gpu-instance-NCt4"
environment_name = "mmrotate-azure-env-gpu"

# Create GPU instance

- https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-compute-instance?view=azureml-api-2&tabs=python

In [ ]:
ci_basic = ComputeInstance(
    name=ci_basic_name,
    size="Standard_NC4as_T4_v3",
    idle_time_before_shutdown_minutes=60,
)
ml_client.begin_create_or_update(ci_basic).result()

# Create Environment

In [44]:
ws = Workspace(
    subscription_id=subscription_id,
    resource_group=resource_group,
    workspace_name=workspace,
)

In [45]:
# https://learn.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment(class)?view=azure-ml-py
azure_env = Environment(name=environment_name)

In [46]:
azure_env = azure_env.from_dockerfile(
    name=environment_name,
    dockerfile="Dockerfile",
)

In [ ]:
response = azure_env.build(ws, image_build_compute=ci_basic_name)

In [ ]:
response.wait_for_completion(show_output=True)

# Submit Verify install job

In [49]:
job = command(
    code="verify_install",  # location of source code
    command="python tester.py",  # script to run
    environment=f"{environment_name}@latest",
    display_name="mmrotate-verify-install",
    # resources=JobResourceConfiguration(
    #     instance_type="Standard_NC4as_T4_v3", instance_count=1
    # ),
    # queue_settings={"job_tier": "spot"},
    compute=ci_basic_name,
)

In [ ]:
ml_client.create_or_update(job)

# Delete GPU instance

In [51]:
ml_client.compute.begin_delete(ci_basic_name).wait()

In [ ]:
print("Finito")